In [1]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import scipy
import scipy.io
import matplotlib.pyplot as plt
import random
import pickle
from torch.utils.data import Dataset, DataLoader
import scipy.signal as sig
from scipy.stats import pearsonr
from utils import *

In [2]:
device = torch.device(0)
device

device(type='cuda', index=0)

In [3]:
raw = scipy.io.loadmat('./datasets/raw_training_data.mat')
data_glove_1 = raw['train_dg'][0][0]
data_glove_1_train = np.delete(data_glove_1, 3, 1)
data_glove_2 = raw['train_dg'][1][0]
data_glove_2_train = np.delete(data_glove_2, 3, 1)
data_glove_3 = raw['train_dg'][2][0]
data_glove_3_train = np.delete(data_glove_3, 3, 1)

ecog_1_train = raw['train_ecog'][0][0]
ecog_2_train = raw['train_ecog'][1][0]
ecog_3_train = raw['train_ecog'][2][0]


raw = scipy.io.loadmat('./datasets/sub1_comp.mat')
ecog_1_comp = raw['train_data']
dg_1_comp = raw['train_dg']
ecog_1_valid = raw['test_data'][49000:]

raw = scipy.io.loadmat('./datasets/sub2_comp.mat')
ecog_2_comp = raw['train_data']
dg_2_comp = raw['train_dg']
ecog_2_valid = raw['test_data'][49000:]

raw = scipy.io.loadmat('./datasets/sub3_comp.mat')
ecog_3_comp = raw['train_data']
dg_3_comp = raw['train_dg']
ecog_3_valid = raw['test_data'][49000:]

dg_1_raw = scipy.io.loadmat('./datasets/sub1_testlabels.mat')
dg_1_valid = dg_1_raw['test_dg'][49000:]
dg_1_valid = np.delete(dg_1_valid, 3, 1)

dg_2_raw = scipy.io.loadmat('./datasets/sub2_testlabels.mat')
dg_2_valid = dg_2_raw['test_dg'][49000:]
dg_2_valid = np.delete(dg_2_valid, 3, 1)

dg_3_raw = scipy.io.loadmat('./datasets/sub3_testlabels.mat')
dg_3_valid = dg_3_raw['test_dg'][49000:]
dg_3_valid = np.delete(dg_3_valid, 3, 1)

In [4]:
winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap

In [5]:
class FingerFeatureDataset(Dataset):
    def __init__(self, R, dg, window=2000):
        self.R = np.float32(R)
#         self.R = (self.R - ecog_1_train.mean(axis=0)) / ecog_1_train.std(axis=0)
#         self.ecog = self.ecog.reshape(self.ecog.shape[0], 1, -1)
        self.dg = np.float32(dg)
        
    

    def __len__(self):
        return len(self.R)

    def __getitem__(self, idx):
        
        return self.R[idx], self.dg[idx]

In [89]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_1_train, 1000, winLen, winOverlap)
# R_train = create_R_matrix(feature_train, 5)

feature_test = get_windowed_feats(ecog_1_valid, 1000, winLen, winOverlap)
# R_test = create_R_matrix(feature_test, 5)

# Downsample the glove data
Y_train = data_glove_1_train
Y_test = dg_1_valid
Y_train = sig.resample(Y_train, feature_train.shape[0], axis=0)
Y_test = sig.resample(Y_test, feature_test.shape[0], axis=0)

R_train = create_R_matrix(feature_train, 20)
R_test = create_R_matrix(feature_test, 20)

idx_1 = feature_selection(R_train, Y_train, 800)
print(idx_1.shape)

R_train = R_train[:, idx_1]
train_mean_S1 = R_train.mean(axis=0)
train_std_S1 = R_train.std(axis=0)
R_train = (R_train - train_mean_S1) / train_std_S1
R_test = R_test[:, idx_1]
R_test = (R_test - train_mean_S1) / train_std_S1

/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 62, using nperseg = 62
  warnings.warn('nperseg = {0:d} is greater than input length '
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: invalid value encountered in divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by z

(615,)


/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: invalid value encountered in divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom


In [90]:
dataset_s1_train = FingerFeatureDataset(R_train.copy(), Y_train.copy())
dataset_s1_valid = FingerFeatureDataset(R_test.copy(), Y_test.copy())

train_loader = DataLoader(dataset_s1_train, batch_size=16, shuffle=False)
test_loader = DataLoader(dataset_s1_valid, batch_size=16, shuffle=False)

In [91]:
class FingerRegressor(nn.Module):
    def __init__(self, num_features, num_fingers) -> None:
        super(FingerRegressor, self).__init__()
        
        self.fc1 = nn.Linear(num_features, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, num_fingers)
        
        self.relu = nn.ReLU()
        
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)
        nn.init.xavier_normal_(self.fc3.weight)

        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.15)
        
    def forward(self, x):

        x = self.dropout1(x)

        
        x = self.relu(self.fc1(x))
        
        x = self.dropout2(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        output = self.fc3(x)

        
        return output

In [94]:
net = FingerRegressor(R_train.shape[1], 4).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())


for epoch in range(30):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pred = []
    for (i, (ecog, dg)) in enumerate(train_loader):
        # print(ecog.shape)
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog)
        pred += [output.detach().cpu().numpy()]
        loss = criterion(output, dg)
        # print (loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    pred = np.concatenate(pred)
    train_cor = correlation_dl(sig.resample(pred, len(data_glove_1_train)), data_glove_1_train)[1]
    train_loss = running_loss / len(train_loader)
    # train_acc = correct / total
    
    net.eval()
    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0
        pred = []
        for (i, (ecog, dg)) in enumerate(test_loader):
            ecog = ecog.to(device)
            dg = dg.to(device)
            output = net(ecog).to(device)
            pred += [output.detach().cpu().numpy()]
            loss = criterion(output, dg)
            running_loss += loss.item()

        pred = np.concatenate(pred)
        val_cor = correlation_dl(sig.resample(pred, len(dg_1_valid)), dg_1_valid)[1]
        
        valid_loss = running_loss / len(test_loader)
    # print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Acc: {train_acc:.3f} | Valid loss: {valid_loss:.3f} | Valid Acc: {val_cor}')
    print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Cor: {train_cor:.3f} | Valid loss: {valid_loss:.3f} | Valid Cor: {val_cor}')
    # break

Epoch 1 | Train loss: 1.504 | Train Cor: 0.101 | Valid loss: 0.964 | Valid Cor: 0.2791198923690979
Epoch 2 | Train loss: 0.947 | Train Cor: 0.252 | Valid loss: 0.909 | Valid Cor: 0.3544631229099592
Epoch 3 | Train loss: 0.855 | Train Cor: 0.333 | Valid loss: 0.836 | Valid Cor: 0.42061805638053645
Epoch 4 | Train loss: 0.794 | Train Cor: 0.398 | Valid loss: 0.819 | Valid Cor: 0.4342122278402871
Epoch 5 | Train loss: 0.756 | Train Cor: 0.434 | Valid loss: 0.790 | Valid Cor: 0.4519443780075736
Epoch 6 | Train loss: 0.726 | Train Cor: 0.460 | Valid loss: 0.767 | Valid Cor: 0.4635174213254888
Epoch 7 | Train loss: 0.696 | Train Cor: 0.484 | Valid loss: 0.751 | Valid Cor: 0.4722486487915055
Epoch 8 | Train loss: 0.693 | Train Cor: 0.487 | Valid loss: 0.757 | Valid Cor: 0.4746569787244396
Epoch 9 | Train loss: 0.670 | Train Cor: 0.507 | Valid loss: 0.731 | Valid Cor: 0.4875335511530971
Epoch 10 | Train loss: 0.658 | Train Cor: 0.519 | Valid loss: 0.728 | Valid Cor: 0.49992471960217727
Epoch 1

In [95]:
np.save('./models/train_mean_S1', train_mean_S1)
np.save('./models/train_std_S1', train_std_S1)
torch.save(net.state_dict(), './models/NN_S1.pth')

In [88]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_2_train, 1000, winLen, winOverlap)
# R_train = create_R_matrix(feature_train, 5)

feature_test = get_windowed_feats(ecog_2_valid, 1000, winLen, winOverlap)
# R_test = create_R_matrix(feature_test, 5)

# Downsample the glove data
Y_train = data_glove_2_train
Y_test = dg_2_valid
Y_train = sig.resample(Y_train, feature_train.shape[0], axis=0)
Y_test = sig.resample(Y_test, feature_test.shape[0], axis=0)

R_train = create_R_matrix(feature_train, 20)
R_test = create_R_matrix(feature_test, 20)

idx_2 = feature_selection(R_train, Y_train, 800)
print(idx_2.shape)

R_train = R_train[:, idx_2]
train_mean_S2 = R_train.mean(axis=0)
train_std_S2 = R_train.std(axis=0)
R_train = (R_train - train_mean_S2) / train_std_S2
R_test = R_test[:, idx_2]
R_test = (R_test - train_mean_S2) / train_std_S2

/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 48, using nperseg = 48
  warnings.warn('nperseg = {0:d} is greater than input length '


KeyboardInterrupt: 

In [63]:
dataset_s2_train = FingerFeatureDataset(R_train.copy(), Y_train.copy())
dataset_s2_valid = FingerFeatureDataset(R_test.copy(), Y_test.copy())

train_loader = DataLoader(dataset_s2_train, batch_size=16, shuffle=False)
test_loader = DataLoader(dataset_s2_valid, batch_size=16, shuffle=False)

In [69]:
net = FingerRegressor(R_train.shape[1], 4).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())


for epoch in range(30):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pred = []
    for (i, (ecog, dg)) in enumerate(train_loader):
        # print(ecog.shape)
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog)
        pred += [output.detach().cpu().numpy()]
        loss = criterion(output, dg)
        # print (loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    pred = np.concatenate(pred)
    train_cor = correlation_dl(sig.resample(pred, len(data_glove_2_train)), data_glove_2_train)[1]
    train_loss = running_loss / len(train_loader)
    # train_acc = correct / total
    
    net.eval()
    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0
        pred = []
        for (i, (ecog, dg)) in enumerate(test_loader):
            ecog = ecog.to(device)
            dg = dg.to(device)
            output = net(ecog).to(device)
            pred += [output.detach().cpu().numpy()]
            loss = criterion(output, dg)
            running_loss += loss.item()

        pred = np.concatenate(pred)
        val_cor = correlation_dl(sig.resample(pred, len(dg_2_valid)), dg_2_valid)[1]
        
        valid_loss = running_loss / len(test_loader)
    # print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Acc: {train_acc:.3f} | Valid loss: {valid_loss:.3f} | Valid Acc: {val_cor}')
    print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Cor: {train_cor:.3f} | Valid loss: {valid_loss:.3f} | Valid Cor: {val_cor}')
    # break

Epoch 1 | Train loss: 1.180 | Train Cor: 0.123 | Valid loss: 0.903 | Valid Cor: 0.22764925068312003
Epoch 2 | Train loss: 0.876 | Train Cor: 0.248 | Valid loss: 0.863 | Valid Cor: 0.2948895611416057
Epoch 3 | Train loss: 0.835 | Train Cor: 0.283 | Valid loss: 0.831 | Valid Cor: 0.3015851873945617
Epoch 4 | Train loss: 0.789 | Train Cor: 0.344 | Valid loss: 0.814 | Valid Cor: 0.34566783621195535
Epoch 5 | Train loss: 0.754 | Train Cor: 0.391 | Valid loss: 0.785 | Valid Cor: 0.3848998740871513
Epoch 6 | Train loss: 0.724 | Train Cor: 0.441 | Valid loss: 0.790 | Valid Cor: 0.3967953174992602
Epoch 7 | Train loss: 0.703 | Train Cor: 0.468 | Valid loss: 0.767 | Valid Cor: 0.4168834035142266
Epoch 8 | Train loss: 0.694 | Train Cor: 0.479 | Valid loss: 0.768 | Valid Cor: 0.422192753143098
Epoch 9 | Train loss: 0.673 | Train Cor: 0.505 | Valid loss: 0.741 | Valid Cor: 0.4234267026077596
Epoch 10 | Train loss: 0.664 | Train Cor: 0.517 | Valid loss: 0.762 | Valid Cor: 0.42633104085426077
Epoch 1

In [70]:
np.save('./models/train_mean_S2', train_mean_S2)
np.save('./models/train_std_S2', train_std_S2)
torch.save(net.state_dict(), './models/NN_S2.pth')

In [96]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_3_train, 1000, winLen, winOverlap)
# R_train = create_R_matrix(feature_train, 5)

feature_test = get_windowed_feats(ecog_3_valid, 1000, winLen, winOverlap)
# R_test = create_R_matrix(feature_test, 5)

# Downsample the glove data
Y_train = data_glove_3_train
Y_test = dg_3_valid
Y_train = sig.resample(Y_train, feature_train.shape[0], axis=0)
Y_test = sig.resample(Y_test, feature_test.shape[0], axis=0)

R_train = create_R_matrix(feature_train, 20)
R_test = create_R_matrix(feature_test, 20)

idx_3 = feature_selection(R_train, Y_train, 800)
print(idx_3.shape)

R_train = R_train[:, idx_3]
train_mean_S3 = R_train.mean(axis=0)
train_std_S3 = R_train.std(axis=0)
R_train = (R_train - train_mean_S3) / train_std_S3
R_test = R_test[:, idx_3]
R_test = (R_test - train_mean_S3) / train_std_S3

/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 64, using nperseg = 64
  warnings.warn('nperseg = {0:d} is greater than input length '
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: invalid value encountered in divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by z

(599,)


/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: invalid value encountered in divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom


In [97]:
dataset_s3_train = FingerFeatureDataset(R_train.copy(), Y_train.copy())
dataset_s3_valid = FingerFeatureDataset(R_test.copy(), Y_test.copy())

train_loader = DataLoader(dataset_s3_train, batch_size=16, shuffle=False)
test_loader = DataLoader(dataset_s3_valid, batch_size=16, shuffle=False)

In [111]:
# net = EEGNet().to(device)
# net = EEGNetRegressor(4).to(device)
# net = EEGNet(n_classes=4, channels=62, samples=3000).to(device) #.cuda(0)
net = FingerRegressor(R_train.shape[1], 4).to(device)
#print (net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)))))#.cuda(0))))
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


for epoch in range(30):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pred = []
    for (i, (ecog, dg)) in enumerate(train_loader):
        # print(ecog.shape)
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog)
        pred += [output.detach().cpu().numpy()]
        loss = criterion(output, dg)
        # print (loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    pred = np.concatenate(pred)
    train_cor = correlation_dl(sig.resample(pred, len(data_glove_3_train)), data_glove_3_train)[1]
    train_loss = running_loss / len(train_loader)
    # train_acc = correct / total
    
    net.eval()
    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0
        pred = []
        for (i, (ecog, dg)) in enumerate(test_loader):
            ecog = ecog.to(device)
            dg = dg.to(device)
            output = net(ecog).to(device)
            pred += [output.detach().cpu().numpy()]
            loss = criterion(output, dg)
            running_loss += loss.item()

        pred = np.concatenate(pred)
        val_cor = correlation_dl(sig.resample(pred, len(dg_3_valid)), dg_3_valid)[1]
        
        valid_loss = running_loss / len(test_loader)
    # print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Acc: {train_acc:.3f} | Valid loss: {valid_loss:.3f} | Valid Acc: {val_cor}')
    print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Cor: {train_cor:.3f} | Valid loss: {valid_loss:.3f} | Valid Cor: {val_cor}')
    # break

Epoch 1 | Train loss: 0.972 | Train Cor: 0.217 | Valid loss: 0.702 | Valid Cor: 0.5027024159343293
Epoch 2 | Train loss: 0.726 | Train Cor: 0.456 | Valid loss: 0.581 | Valid Cor: 0.6261185812030904
Epoch 3 | Train loss: 0.601 | Train Cor: 0.573 | Valid loss: 0.522 | Valid Cor: 0.6732182618683221
Epoch 4 | Train loss: 0.547 | Train Cor: 0.622 | Valid loss: 0.496 | Valid Cor: 0.6950261718740653
Epoch 5 | Train loss: 0.520 | Train Cor: 0.644 | Valid loss: 0.490 | Valid Cor: 0.7044172981382085
Epoch 6 | Train loss: 0.494 | Train Cor: 0.668 | Valid loss: 0.482 | Valid Cor: 0.7091621003933387
Epoch 7 | Train loss: 0.476 | Train Cor: 0.684 | Valid loss: 0.474 | Valid Cor: 0.7155927608430317
Epoch 8 | Train loss: 0.466 | Train Cor: 0.690 | Valid loss: 0.463 | Valid Cor: 0.7178863615545731
Epoch 9 | Train loss: 0.466 | Train Cor: 0.691 | Valid loss: 0.471 | Valid Cor: 0.7157061504442087
Epoch 10 | Train loss: 0.450 | Train Cor: 0.703 | Valid loss: 0.468 | Valid Cor: 0.7182267598521104
Epoch 11 

In [112]:
np.save('./models/train_mean_S3', train_mean_S3)
np.save('./models/train_std_S3', train_std_S3)
torch.save(net.state_dict(), './models/NN_S3.pth')

In [115]:
ecog_1_leaderboard = ecog_1_comp[500: 500 + 147500]
dg_1_leaderboard = dg_1_comp[500: 500 + 147500]

ecog_2_leaderboard = ecog_2_comp[500: 500 + 147500]
dg_2_leaderboard = dg_2_comp[500: 500 + 147500]

ecog_3_leaderboard = ecog_3_comp[500: 500 + 147500]
dg_3_leaderboard = dg_3_comp[500: 500 + 147500]

winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap


feature_1 = get_windowed_feats(ecog_1_leaderboard, 1000, winLen, winOverlap)
# R_1 = create_R_matrix(feature_1, 5)
feature_2 = get_windowed_feats(ecog_2_leaderboard, 1000, winLen, winOverlap)
# R_2 = create_R_matrix(feature_2, 5)
feature_3 = get_windowed_feats(ecog_3_leaderboard, 1000, winLen, winOverlap)
# R_3 = create_R_matrix(feature_3, 5)

# idx_1 = np.load('./models/idx_S1.npy')
# idx_2 = np.load('./models/idx_S2.npy')
# idx_3 = np.load('./models/idx_S3.npy')

R_1 = create_R_matrix(feature_1, 20)[:, idx_1]
R_2 = create_R_matrix(feature_2, 20)[:, idx_2]
R_3 = create_R_matrix(feature_3, 20)[:, idx_3]


/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 62, using nperseg = 62
  warnings.warn('nperseg = {0:d} is greater than input length '
/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 48, using nperseg = 48
  warnings.warn('nperseg = {0:d} is greater than input length '
/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 64, using nperseg = 64
  warnings.warn('nperseg = {0:d} is greater than input length '


In [116]:
train_mean_S1 = np.load('./models/train_mean_S1.npy')
train_std_S1 = np.load('./models/train_std_S1.npy')

train_mean_S2 = np.load('./models/train_mean_S2.npy')
train_std_S2 = np.load('./models/train_std_S2.npy')

train_mean_S3 = np.load('./models/train_mean_S3.npy')
train_std_S3 = np.load('./models/train_std_S3.npy')

In [117]:
leaderboard_dataset = FingerFeatureDataset((R_1 - train_mean_S1) / train_std_S1, np.zeros(R_1.shape[0]).copy())
dataloader = DataLoader(leaderboard_dataset, batch_size=16, shuffle=False)

net = FingerRegressor(R_1.shape[1], 4).to(device)
net.load_state_dict(torch.load('./models/NN_S1.pth'))

with torch.no_grad():
    pred = []
    net.eval()
    for i, (ecog, dg) in enumerate(dataloader):
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog).to(device)
        pred += [output.detach().cpu().numpy()]

    pred = np.concatenate(pred)
correlation_dl(sig.resample(pred, len(dg_1_leaderboard)), dg_1_leaderboard)[1]

0.5553528686861728

In [118]:
leaderboard_dataset = FingerFeatureDataset((R_2 - train_mean_S2) / train_std_S2, np.zeros(R_2.shape[0]).copy())
dataloader = DataLoader(leaderboard_dataset, batch_size=16, shuffle=False)

net = FingerRegressor(R_2.shape[1], 4).to(device)
net.load_state_dict(torch.load('./models/NN_S2.pth'))

with torch.no_grad():
    pred = []
    net.eval()
    for i, (ecog, dg) in enumerate(dataloader):
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog).to(device)
        pred += [output.detach().cpu().numpy()]

    pred = np.concatenate(pred)
correlation_dl(sig.resample(pred, len(dg_2_leaderboard)), dg_2_leaderboard)[1]

0.5654337122490332

In [119]:
leaderboard_dataset = FingerFeatureDataset((R_3 - train_mean_S3) / train_std_S3, np.zeros(R_3.shape[0]).copy())
dataloader = DataLoader(leaderboard_dataset, batch_size=16, shuffle=False)

net = FingerRegressor(R_3.shape[1], 4).to(device)
net.load_state_dict(torch.load('./models/NN_S3.pth'))

with torch.no_grad():
    pred = []
    net.eval()
    for i, (ecog, dg) in enumerate(dataloader):
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog).to(device)
        pred += [output.detach().cpu().numpy()]

    pred = np.concatenate(pred)
correlation_dl(sig.resample(pred, len(dg_3_leaderboard)), dg_3_leaderboard)[1]

0.6924273174017038